In [8]:
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

In [9]:
line_file= gpd.read_file('test_file.geojson')

In [10]:
print(line_file)

   id                                           geometry
0   1  MULTILINESTRING ((83.95616 28.21403, 83.96005 ...
1   2  MULTILINESTRING ((83.98304 28.20999, 83.98470 ...


In [12]:
line_file = line_file.to_crs(epsg=32645)
line_file['length']= line_file.geometry.length

In [13]:
print(line_file)

   id                                           geometry       length
0   1  MULTILINESTRING ((201237.036 3124666.070, 2016...  6123.115805
1   2  MULTILINESTRING ((203866.130 3124152.274, 2040...  1606.282434
